In [14]:
from environment import Environment
from constraints import ForwardConstraints
from trainer import Trainer
from pathlib import Path
from main import set_seed
import os
import json
import argparse
import torch
from typing import Callable
device = torch.device('cpu')

# Get cmd line arguments to reconstruct model
result_path = "result/diagonal_5x5_20220718_17_39_28.988157"
with open(os.path.join(result_path, "args.txt"), "r") as f:
    args_dict = json.loads(f.read())
args = argparse.Namespace(**args_dict)
print(args_dict)

environment = environment = Environment(Path(f"./environments/{args.environment}"), groups_file=args.groups_file)

{'hidden_size': 128, 'static_size': 2, 'dynamic_size': 1, 'num_layers': 1, 'dropout': 0.1, 'seed': None, 'checkpoint': None, 'test': False, 'epoch_max': 10, 'train_size': 128, 'line_unit_price': 1.0, 'station_price': 5.0, 'result_path': None, 'actor_lr': 0.001, 'critic_lr': 0.001, 'actor_mlp_layers': 5, 'critic_mlp_layers': 4, 'station_num_lim': 45, 'budget': 210, 'max_grad_norm': 2.0, 'environment': 'diagonal_5x5', 'reward': 'weighted', 'ses_weight': 0, 'var_lambda': 0, 'groups_file': None, 'arch': 'pointer'}
No groups file provided. Trying to use the default groups file.


In [15]:
# Load environment, constraints, and trainer
if args.seed:
    set_seed(args.seed)
environment = Environment(Path(f"./environments/{args.environment}"), groups_file=args.groups_file)
constraints = ForwardConstraints(environment.grid_x_size, environment.grid_y_size, environment.existing_lines_full, environment.grid_to_vector)
trainer = Trainer(environment, constraints, args)
trainer.load_checkpoint(result_path)

No groups file provided. Trying to use the default groups file.
Number of trainable parameters actor-critic: 182912 / 139181


In [16]:
static = environment.static.to(device)
dynamic = torch.zeros((1, args.dynamic_size, environment.grid_size)).float().to(device)

tour_idx = trainer.actor(static, dynamic, args.station_num_lim, budget=args.budget,
                                                 line_unit_price=args.line_unit_price, station_price=args.station_price,
                                                 decoder_input=None, last_hh=None)

/Users/lo/git/fair-network-expansion/environment.py:58: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  grid_x = (vector_idx // self.grid_y_size)


In [17]:
fn1 = lambda d_eucl, d_tour: torch.nn.functional.relu(d_eucl/d_tour)
fn2 = lambda d_eucl, d_tour: torch.nn.functional.relu(1 - ((d_tour-1)/d_eucl))

def matrix_reward_scaling(tour_idx: torch.Tensor, environment: Environment, scaling_fn: Callable):
    tour_idx = tour_idx[0].flatten()
    tour_dict = {(s1, s2): tour_idx[i:j+1] for i, s1 in enumerate(tour_idx) for j, s2 in enumerate(tour_idx) if i != j and i < j}
    od_scaling_mask = torch.zeros(environment.grid_size, environment.grid_size, device=device)

    for c, subtour in tour_dict.items():
        # Calculate direct euclidian distance
        p1 = environment.vector_to_grid(c[0]).float()
        p2 = environment.vector_to_grid(c[1]).float()
        d_eucl = torch.dist(p1, p2, p=2)

        # Calculate distance via subtour
        points = [environment.vector_to_grid(p).float() for p in subtour]
        d_tour = sum([torch.dist(points[i], points[i+1]) for i, _ in enumerate(points) if i != len(points) - 1])

        # Calculate scaling factor for subtour connection
        scaled = scaling_fn(d_eucl, d_tour)
        od_scaling_mask[c[0], c[1]] = scaled
        
    
    return od_scaling_mask
    
print(tour_idx)
print(matrix_reward_scaling(tour_idx, environment, fn1))

(tensor([[24, 12,  2,  1,  0]]), tensor([[-3.2430e+00, -2.0702e+00, -2.0905e+00, -6.9951e-01, -1.1921e-07]],
       grad_fn=<CatBackward0>))
torch.Size([25, 25])
tensor([[1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 

/Users/lo/git/fair-network-expansion/environment.py:58: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  grid_x = (vector_idx // self.grid_y_size)


In [18]:
a = torch.Tensor([0,1,2,3,4])
tour_dict = {(s1, s2): a[i:j+1] for i, s1 in enumerate(a) for j, s2 in enumerate(a) if i != j and i < j}
for i, s1 in enumerate(a):
    for j, s2 in enumerate(a):
        if i != j and i < j:
            print((s1,s2), a[i:j+1])
print(tour_dict)

(tensor(0.), tensor(1.)) tensor([0., 1.])
(tensor(0.), tensor(2.)) tensor([0., 1., 2.])
(tensor(0.), tensor(3.)) tensor([0., 1., 2., 3.])
(tensor(0.), tensor(4.)) tensor([0., 1., 2., 3., 4.])
(tensor(1.), tensor(2.)) tensor([1., 2.])
(tensor(1.), tensor(3.)) tensor([1., 2., 3.])
(tensor(1.), tensor(4.)) tensor([1., 2., 3., 4.])
(tensor(2.), tensor(3.)) tensor([2., 3.])
(tensor(2.), tensor(4.)) tensor([2., 3., 4.])
(tensor(3.), tensor(4.)) tensor([3., 4.])
{(tensor(0.), tensor(1.)): tensor([0., 1.]), (tensor(0.), tensor(2.)): tensor([0., 1., 2.]), (tensor(0.), tensor(3.)): tensor([0., 1., 2., 3.]), (tensor(0.), tensor(4.)): tensor([0., 1., 2., 3., 4.]), (tensor(1.), tensor(2.)): tensor([1., 2.]), (tensor(1.), tensor(3.)): tensor([1., 2., 3.]), (tensor(1.), tensor(4.)): tensor([1., 2., 3., 4.]), (tensor(2.), tensor(3.)): tensor([2., 3.]), (tensor(2.), tensor(4.)): tensor([2., 3., 4.]), (tensor(3.), tensor(4.)): tensor([3., 4.])}


In [23]:
a = torch.zeros(3,3).byte()
a[1,1] = 1
a

tensor([[0, 0, 0],
        [0, 1, 0],
        [0, 0, 0]], dtype=torch.uint8)

In [24]:
a.any(dim=0)

tensor([0, 1, 0], dtype=torch.uint8)